# Beautiful Soup实战-爬取豆瓣top250图书

推文链接：[BeautifulSoup库实战-爬取豆瓣top250图书](https://www.jianshu.com/p/dfaa49c03b33)

In [2]:
#导入相关库
import re
import requests
from bs4 import BeautifulSoup

### 一、得到所有网址链接

In [65]:
def allurl():
    html = []
    for i in range(10):
        url = 'https://book.douban.com/top250?start='+str(i*25)
        html.append(url)
    return html

In [3]:
allurl()

['https://book.douban.com/top250?start=0',
 'https://book.douban.com/top250?start=25',
 'https://book.douban.com/top250?start=50',
 'https://book.douban.com/top250?start=75',
 'https://book.douban.com/top250?start=100',
 'https://book.douban.com/top250?start=125',
 'https://book.douban.com/top250?start=150',
 'https://book.douban.com/top250?start=175',
 'https://book.douban.com/top250?start=200',
 'https://book.douban.com/top250?start=225']

### 二、获取网页源代码内容

In [3]:
book_url = 'https://book.douban.com/top250?start='
book_info = requests.get(book_url,timeout = 30) #使用requests返回网页的整体结构
book_info.text

'\n\n<!DOCTYPE html>\n<html lang="zh-cmn-Hans" class=" book-new-nav">\n<head>\n  <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n  <title>豆瓣图书 Top 250</title>\n  \n<script>!function(e){var o=function(o,n,t){var c,i,r=new Date;n=n||30,t=t||"/",r.setTime(r.getTime()+24*n*60*60*1e3),c="; expires="+r.toGMTString();for(i in o)e.cookie=i+"="+o[i]+c+"; path="+t},n=function(o){var n,t,c,i=o+"=",r=e.cookie.split(";");for(t=0,c=r.length;t<c;t++)if(n=r[t].replace(/^\\s+|\\s+$/g,""),0==n.indexOf(i))return n.substring(i.length,n.length).replace(/\\"/g,"");return null},t=e.write,c={"douban.com":1,"douban.fm":1,"google.com":1,"google.cn":1,"googleapis.com":1,"gmaptiles.co.kr":1,"gstatic.com":1,"gstatic.cn":1,"google-analytics.com":1,"googleadservices.com":1},i=function(e,o){var n=new Image;n.onload=function(){},n.src="https://www.douban.com/j/except_report?kind=ra022&reason="+encodeURIComponent(e)+"&environment="+encodeURIComponent(o)},r=function(o){try{t.call(e,o)}catch(e){t(o)}

### 三、提取所需信息

In [6]:
soup = BeautifulSoup(book_info.text, 'lxml') # 使用lxml作为解析器，返回一个Beautifulsoup对象
table = soup.findAll('table', {"width": "100%"}) #找到其中所有width=100%的table标签），即找到所有的书

In [8]:
table

[<table width="100%">
 <tr class="item">
 <td valign="top" width="100">
 <a class="nbg" href="https://book.douban.com/subject/1770782/" onclick="moreurl(this,{i:'0'})">
 <img src="https://img3.doubanio.com/view/subject/m/public/s1727290.jpg" width="90"/>
 </a>
 </td>
 <td valign="top">
 <div class="pl2">
 <a href="https://book.douban.com/subject/1770782/" onclick="&quot;moreurl(this,{i:'0'})&quot;" title="追风筝的人">
                 追风筝的人
 
                 
               </a>
 
 
 
                   <img alt="可试读" src="https://img3.doubanio.com/pics/read.gif" title="可试读"/>
 <br/>
 <span style="font-size:12px;">The Kite Runner</span>
 </div>
 <p class="pl">[美] 卡勒德·胡赛尼 / 李继宏 / 上海人民出版社 / 2006-5 / 29.00元</p>
 <div class="star clearfix">
 <span class="allstar45"></span>
 <span class="rating_nums">8.9</span>
 <span class="pl">(
                     401356人评价
                 )</span>
 </div>
 <p class="quote" style="margin: 10px 0; color: #666">
 <span class="inq">为你，千千万万遍</span>
 </p>
 </td

In [12]:
#测试
cs = table[0]

In [30]:
#获取书名 
name = cs.div.a.get_text()#获取标签里内容
name

'\n                追风筝的人\n\n                \n              '

In [31]:
name = name.strip()#清除空格、换行符
name

'追风筝的人'

In [33]:
#获取评分
score = cs.find('span', {"class": "rating_nums"}).text.strip() #获取分数
score

'8.9'

In [34]:
#获取评价人数
nums = cs.find('span', {"class": "pl"}).text.strip() # 获取评价人数
num = re.findall('(\d+)人评价', nums)[0]  # 通过正则取具体的数字
num

'401356'

In [44]:
#存储在字典中
data = {'title': name, 'rate': score, 'vote': num}
data

{'title': '追风筝的人', 'rate': '8.9', 'vote': '401356'}

In [118]:
#构造函数book()获取所有图书相关信息
def book(target_url):
    books = []
    book_info = requests.get(target_url)
    soup = BeautifulSoup(book_info.text, 'lxml') 
    table = soup.findAll('table', {"width": "100%"})
    for item in table:
        name = item.div.a.get_text().strip() #获取书名
        r_name = name.replace('\n', '').replace(' ', '') #通过看网页的HTML结构，可以发现书名后是有换行以及空格的，将这些全部通过replace替换去除
        score = item.find('span', {"class": "rating_nums"}).text.strip() #获取分数    
        nums = item.find('span', {"class": "pl"}).text.strip() # 获取评价人数
        num = re.findall('(\d+)人评价', nums)[0]  # 通过正则取具体的数字
        books.append((r_name, score, num)) #以元组存入列表
    return books #返回一页的书籍

In [119]:
book("https://book.douban.com/top250?start=")

[('追风筝的人', '8.9', '401399'),
 ('解忧杂货店', '8.5', '384494'),
 ('小王子', '9.0', '324296'),
 ('白夜行', '9.1', '283605'),
 ('围城', '8.9', '243683'),
 ('三体:“地球往事”三部曲之一', '8.8', '233962'),
 ('嫌疑人X的献身', '8.9', '229671'),
 ('挪威的森林', '8.0', '230926'),
 ('活着', '9.3', '175474'),
 ('百年孤独', '9.2', '175284'),
 ('红楼梦', '9.6', '167129'),
 ('看见', '8.8', '170331'),
 ('不能承受的生命之轻', '8.5', '159223'),
 ('平凡的世界（全三部）', '9.0', '143651'),
 ('三体Ⅱ:黑暗森林', '9.3', '137335'),
 ('达·芬奇密码', '8.2', '154760'),
 ('三体Ⅲ:死神永生', '9.2', '136118'),
 ('我们仨', '8.7', '126938'),
 ('简爱（英文全本）', '8.5', '131044'),
 ('天才在左疯子在右:国内第一本精神病人访谈手记', '8.2', '135674'),
 ('哈利·波特与魔法石', '9.0', '114124'),
 ('傲慢与偏见', '8.8', '114036'),
 ('明朝那些事儿（壹）:洪武大帝', '8.7', '113710'),
 ('目送', '8.7', '114187'),
 ('飘', '9.3', '100526')]

### 四、合并函数并导出表格

In [120]:
import pandas as pd
from pandas import DataFrame
df = {'title':[],'rate':[],'vote':[]}
all = DataFrame(df)

In [121]:
for url in allurl():
    frame = DataFrame(book(url),columns=['title', 'rate', 'vote'])
    all = pd.concat([all,frame],ignore_index=True)

In [122]:
all

,title,rate,vote
0,追风筝的人,8.9,401399
1,解忧杂货店,8.5,384494
2,小王子,9.0,324296
3,白夜行,9.1,283605
4,围城,8.9,243683
5,三体:“地球往事”三部曲之一,8.8,233962
6,嫌疑人X的献身,8.9,229671
7,挪威的森林,8.0,230926
8,活着,9.3,175474
9,百年孤独,9.2,175284


In [124]:
##写入到excel
all.to_csv('D:/data/2019/bs4-booktop250.csv', sep=',', header=True, index=True,encoding='utf-8')